In [ ]:
# licensed under the Creative Commons - Attribution-NonCommercial 4.0
# International license (CC BY-NC 4.0):
# https://creativecommons.org/licenses/by-nc/4.0/. 

In [ ]:
## enc-dec architecture, without error variance

## models are saved in snapshots folder
## this fn returns forecasts
def train_model(model_name, iterations, vals_train, static_cat, lookback, loss_fn_name, use_static_cat, targ_name, exog_names, horizon, use_norm=True):

    input_size = lookback * horizon  ## backward window size, in time units
    ts_freq = 1 ## only needed for MASE loss
    history_size_in_horizons = 60 ## model only uses this much history, max
    use_vars = [targ_name, *exog_names]
    
    training_values = np.stack(
                        [vals_train[k] for k in use_vars]
                        ).transpose([1,2,0]) ## dims are [series, time, variables]
                                            ## where the first variable is the target, and the rest are covariates
    
    n_features = training_values.shape[2] - 1 ## number of exogenous covariates
    n_embed = static_cat.shape[0]

    embed_dim = 2

    ## TCN receptive field size = 2(k-1)(2^n - 1)+1 where k is kernel size and n is # blocks
    tcn_k = 3
    tcn_n = 4
    tcn_dropout = 0.0
    enc_hid = 128
    enc_temporal = True
    enc_dim = input_size if enc_temporal else enc_hid

    if use_static_cat:
        #exog_block = LSTM_back_cat(n_features=n_features,input_size=input_size,output_size=horizon,
        #            layer_size=2048,n_embed=n_embed,embed_dim=embed_dim,decoder_extra_layers=0,lstm_layers=2,lstm_hidden=256)
        #tcn_chans = n_features + embed_dim + 1
        #exog_block = TCN(n_features, input_size, horizon, [enc_hid]*tcn_n, tcn_k, tcn_dropout, enc_temporal, n_embed, embed_dim)
        exog_block = TCN_encoder(n_features, [enc_hid]*tcn_n, tcn_k, tcn_dropout, enc_temporal, n_embed, embed_dim)
        #exog_block = LSTM_test(n_features=n_features,input_size=input_size,output_size=horizon,layer_size=-1,n_embed=n_embed,embed_dim=embed_dim,decoder_extra_layers=0,
        #            lstm_layers=1,lstm_hidden=enc_hid,temporal=enc_temporal,decode=False)
    else:
        #exog_block = LSTM_with_backcast(n_features=n_features,input_size=input_size,output_size=horizon,
        #            layer_size=2048,decoder_extra_layers=0,lstm_layers=2,lstm_hidden=256)
        #tcn_chans = n_features + 1
        #exog_block = TCN(n_features, input_size, horizon, [enc_hid]*tcn_n, tcn_k, tcn_dropout, enc_temporal)
        exog_block = TCN_encoder(n_features, [enc_hid]*tcn_n, tcn_k, tcn_dropout, enc_temporal)
        #exog_block = LSTM_test(n_features=n_features,input_size=input_size,output_size=horizon,layer_size=-1,decoder_extra_layers=0,
        #            lstm_layers=1,lstm_hidden=enc_hid,temporal=enc_temporal,decode=False)
    
    #model = generic_exog(input_size=input_size, output_size=horizon,
    #                stacks = 8,
    #                layers = 4,  ## 4 per stack, from the paper
    #                layer_size = -1, #128,
    #                exog_block = exog_block,
    #                exog_first=True,use_norm=use_norm,
    #                exog_only=True)
    
    model = generic_decoder(enc_dim=enc_dim, output_size=horizon,
                    stacks = 8,
                    layers = 4,  ## 4 per stack, from the paper
                    layer_size = 128,
                    exog_block = exog_block,
                    use_norm=use_norm)
    
    train_ds = ts_dataset(timeseries=training_values, static_cat=static_cat,
                                    insample_size=input_size,
                                    outsample_size=horizon,
                                    window_sampling_limit=int(history_size_in_horizons * horizon))

    ##
    ## batch size?
    ##
    training_set = DataLoader(train_ds,batch_size=128)#1024)

    snapshot_manager = SnapshotManager(snapshot_dir=os.path.join('hub_model_snapshots', model_name),
                                        total_iterations=iterations)

    model = trainer(snapshot_manager=snapshot_manager,
                    model=model,
                    training_set=iter(training_set),
                    timeseries_frequency=ts_freq,  ## only used by MASE loss fn
                    loss_name=loss_fn_name,
                    iterations=iterations)
    
    # Build forecasts
    x, x_mask, cat_tensor = train_ds.last_insample_window()
    model.eval()
    forecast = []
    with t.no_grad():
        forecast.extend(model(x, x_mask, cat_tensor).cpu().detach().numpy())
    
    return forecast




In [ ]:
use_norm = True
var_transform = not use_norm

## log transform target?
if var_transform:
    vals_train[targ_var] = np.log(vals_train[targ_var] + 1.0)
    vals_train["future_vals"] = np.log(vals_train["future_vals"] + 1.0)


forecasts = {}
for i,lookback in enumerate(lookback_opts):
    for j,use_static_cat in enumerate(use_cat_opts):
        for k,loss_fn_name in enumerate(loss_fn_opts):
            model_name = "model"+str(i)+str(j)+str(k)
            print("training ",model_name)
            forecasts[model_name] = train_model(model_name,iterations,
                                                vals_train,static_cat,
                                                lookback,loss_fn_name,use_static_cat,
                                                targ_var,exog_vars,
                                                horizon,use_norm)
            
## ensemble using median
forecasts["median"] = np.median(np.stack([np.stack(forecasts[k]) for k in forecasts]),axis=0)

## forecasts to natural scale
if var_transform:
    vals_train[targ_var] = np.exp(vals_train[targ_var]) - 1.0
    vals_train["future_vals"] = np.exp(vals_train["future_vals"]) - 1.0
    for k in forecasts:
        forecasts[k] = [np.exp(v) - 1.0 for v in forecasts[k]]

upper_fc, lower_fc = None, None

In [ ]:
## univariate, without error var

def train_univar(model_name, iterations, training_values, lookback, loss_fn_name, use_norm, horizon,
                 stacks = 8, layer_size = 128):

    horizon = horizon ## forward window size, in time units (weeks)
    input_size = lookback * horizon  ## backward window size, in time units
    ts_freq = 1 ## only needed for MASE loss
    history_size_in_horizons = 60 ## model only uses this much history, max

    model = generic(input_size=input_size, output_size=horizon,
                    stacks = stacks,
                    layers = 4,  ## per stack, from the paper
                    layer_size = layer_size,
                    use_norm=use_norm, 
                    dropout=None)
    
    train_ds = ts_dataset(timeseries=training_values, static_cat=None,
                                    insample_size=input_size,
                                    outsample_size=horizon,
                                    window_sampling_limit=int(history_size_in_horizons * horizon))

    training_set = DataLoader(train_ds,batch_size=256)#1024)  ######

    snapshot_manager = SnapshotManager(snapshot_dir=os.path.join('hub_model_snapshots', model_name),
                                        total_iterations=iterations)

    model = trainer(snapshot_manager=snapshot_manager,
                    model=model,
                    training_set=iter(training_set),
                    timeseries_frequency=ts_freq,  ## only used by MASE loss fn
                    loss_name=loss_fn_name,
                    iterations=iterations)
    
    # Build forecasts
    x, x_mask, cat_tensor = train_ds.last_insample_window()
    model.eval()
    forecast = []
    with t.no_grad():
        forecast.extend(model(x, x_mask, cat_tensor).cpu().detach().numpy())
    
    return forecast


In [ ]:
## univariate, with error var

def train_u_var(model_name, iterations, train_mu, lookback, loss_fn_name, use_norm, horizon,
                stacks = 8, layer_size = 128):

    horizon = horizon ## forward window size, in time units (weeks)
    input_size = lookback * horizon  ## backward window size, in time units
    history_size_in_horizons = 60 ## model only uses this much history, max

    model = generic_var(input_size=input_size, output_size=horizon,
                    stacks = stacks,
                    layers = 4,  ## per stack, from the paper
                    layer_size = layer_size)
    
    train_ds = ts_dataset(timeseries=train_mu, static_cat=None, insample_size=input_size, outsample_size=horizon,
                                    window_sampling_limit=int(history_size_in_horizons * horizon))

    training_set = DataLoader(train_ds,batch_size=1024)

    snapshot_manager = SnapshotManager(snapshot_dir=os.path.join('hub_model_snapshots', model_name),
                                        total_iterations=iterations)

    model = trainer_var(snapshot_manager=snapshot_manager,
                    model=model,
                    training_set=iter(training_set),
                    timeseries_frequency=0,  ## not used
                    loss_name=loss_fn_name,
                    iterations=iterations)
    
    # Build forecasts
    x, x_mask, cat_tensor = train_ds.last_insample_window()
    model.eval()
    forecast_mu = []
    forecast_var = []
    with t.no_grad():
        f_mu, f_var = model(x, x_mask, cat_tensor)
        forecast_mu.extend(f_mu.cpu().detach().numpy())
        forecast_var.extend(f_var.cpu().detach().numpy())
    
    return forecast_mu, forecast_var


In [ ]:
## weekly data

cut = 140

vals_train = {}
targ_var = "h_mean"

## load target files
df_targ = pd.read_csv("storage/training_data/"+targ_var+"_weekly.csv",index_col=0).iloc[:cut,:]
vals_train[targ_var] = df_targ.to_numpy(dtype=np.float32).transpose() ## dims are [series, time]

df_targ_all = pd.read_csv("storage/training_data/"+targ_var+"_weekly.csv",index_col=0)
if cut is not None:
    test_targets = df_targ_all.iloc[cut:,:].to_numpy(dtype=np.float32).transpose()
else:
    test_targets = None

iterations = 100


forecasts = {}
lookback_opts = [4,6,8]
horizon_opts = [7]
loss_fn_opts = ["MAPE"]#,"SMAPE"]
for i,lookback in enumerate(lookback_opts):
    for j,horizon in enumerate(horizon_opts):
        for k,loss_fn_name in enumerate(loss_fn_opts):
            model_name = "univar_"+str(i)+str(j)+str(k)
            use_windowed_norm = True

            if use_windowed_norm:
                t_vals = vals_train[targ_var]
            else:
                u = np.nanmean(vals_train[targ_var],axis=1,keepdims=True)
                s = np.nanstd(vals_train[targ_var],axis=1,keepdims=True)
                t_vals = (vals_train[targ_var] - u) / s 

            print("training ",model_name)
            f_vals = train_univar(model_name,iterations,
                                                t_vals,
                                                lookback,loss_fn_name,
                                                use_windowed_norm,horizon)
            if use_windowed_norm:
                forecasts[model_name] = f_vals
            else:
                forecasts[model_name] = u + f_vals * s
            
## ensemble using median
forecasts["median"] = np.median(np.stack([np.stack(forecasts[k]) for k in forecasts]),axis=0)


print("cut", cut)
for j,horizon in enumerate(horizon_opts):
    for i,lookback in enumerate(lookback_opts):
        print("lookback",lookback,"horizon",horizon,end=" ")
        for k,loss_fn_name in enumerate(loss_fn_opts):
            model_name = "univar_"+str(i)+str(j)+str(k)
            print(calc_loss(forecasts, model_name, test_targets, horizon),end=" ")
        print(" ")

In [ ]:
## daily data

cut = None#165 * 7

vals_train = {}
targ_var = "h"

df_targ_all = pd.read_csv("storage/training_data/"+targ_var+"_7ma.csv",index_col=0)
df_targ = df_targ_all.iloc[:cut,:]
vals_train[targ_var] = df_targ.to_numpy(dtype=np.float32).transpose() ## dims are [series, time]

if cut is not None:
    test_targets = df_targ_all.iloc[cut:,:].to_numpy(dtype=np.float32).transpose()
else:
    test_targets = None

iterations = 50


In [ ]:
## without error var

iterations = 60

lookback_opts = [4,6,8]
loss_fn_name = "MAPE"
horizon = 40 
stacks = 12
layer_size = 512

forecasts = {}
for i,lookback in enumerate(lookback_opts):
    model_name = "univar_"+str(lookback)
    print("training ",model_name)
    forecasts[model_name] = train_univar(model_name,iterations,
                                        vals_train[targ_var],
                                        lookback,loss_fn_name,
                                        True,
                                        horizon,stacks,layer_size)

## ensemble using median
forecasts["univar_med"] = np.median(np.stack([np.stack(forecasts[k]) for k in forecasts]),axis=0)



In [ ]:
## with error var

#iterations = 100

var_transform = True

## log transform target?
if var_transform:
    targ_vals = np.log(vals_train[targ_var] + 1.0)
    #targ_vals = np.apply_along_axis(lambda s: stats.boxcox(s+1.0)[0], 1, vals_train[targ_var])
    #lambdas = np.apply_along_axis(lambda s: stats.boxcox(s+1.0)[1], 1, vals_train[targ_var])
else:
    targ_vals = vals_train[targ_var]

mu_fc={}
var_fc={}
for i,lookback in enumerate(lookback_opts):
    model_name = "u_var_"+str(lookback)
    print("training ",model_name)
    mu_fc[model_name], var_fc[model_name] = train_u_var(model_name,iterations,
                                                        targ_vals,
                                                        lookback,"",
                                                        False,
                                                        horizon,stacks,layer_size)

## ensemble using median
mu_fc["u_var_med"] = np.median(np.stack([np.stack(mu_fc[k]) for k in mu_fc]),axis=0)
var_fc["u_var_med"] = np.median(np.stack([np.stack(var_fc[k]) for k in var_fc]),axis=0)

std_fc = {}
upper_fc = {}
lower_fc = {}
for k in mu_fc:
    std_fc[k] = [np.sqrt(v) for v in var_fc[k]]
    upper_fc[k] = [mu_fc[k][i] + 2.0*v for (i,v) in enumerate(std_fc[k])]
    lower_fc[k] = [mu_fc[k][i] - 2.0*v for (i,v) in enumerate(std_fc[k])]


## forecasts to natural scale
if var_transform:
    for k in mu_fc:
        mu_fc[k] = [np.exp(v) - 1.0 for v in mu_fc[k]]
        upper_fc[k] = [np.exp(v) - 1.0 for v in upper_fc[k]]    
        lower_fc[k] = [np.exp(v) - 1.0 for v in lower_fc[k]]    
        #mu_fc[k] = [inv_boxcox(v,lambdas[i]) - 1.0 for (i,v) in enumerate(mu_fc[k])]


In [ ]:

forecasts = forecasts | mu_fc

print("cut", cut)
for i,lookback in enumerate([*lookback_opts, "med"]):
    print("lookback",lookback,end=" ")
    for model_name in ["univar_"+str(lookback), "u_var_"+str(lookback)]:
        print(calc_loss(forecasts, model_name, test_targets, horizon),end=" ")
    print(" ")

In [ ]:
snapshot_manager = SnapshotManager(snapshot_dir=os.path.join('hub_model_snapshots', "univar_100"), total_iterations=iterations)

ldf = snapshot_manager.load_training_losses()
_, ax = plt.subplots(figsize=[4,3])
ax.plot(ldf)
plt.show()

In [ ]:
plotpred(forecasts, "u_var_med", 20, vals_train[targ_var], test_targets, horizon, lower_fc, upper_fc)

In [ ]:
plotpred(forecasts, "univar_000", 4, vals_train[targ_var], test_targets, horizon)